<a href="https://colab.research.google.com/github/being3iimed/Tanitjobs-data/blob/main/FScrappingTanitjobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Scrapping Stage**

Imports

In [ ]:
import requests
import time
import csv
import re
from bs4 import BeautifulSoup as bs


Writing to csv file

In [ ]:
class Writer :
  #todo
  def __init__(self, filename) :
    self.filename = str(filename)

  def write_to_csv(self, list_input):
      # The scraped info will be written to a CSV here.
      try:
          with open(self.filename, encoding='utf-8-sig', mode="a", newline="") as fopen:  # Open the csv file.
              csv_writer = csv.writer(fopen)
              csv_writer.writerow(list_input)
      except:
          return False

In [ ]:
!pip install googletrans==4.0.0-rc1 q
from googletrans import Translator

In [ ]:
# Function to translate text with exception handling
def translate_text(text, source_lang, target_lang='en'):
    translator = Translator()
    try:
        translation = translator.translate(text, src=source_lang, dest=target_lang)
        return translation.text
    except Exception as e:
        print(f"Translation failed for text: {text}. Error: {e}")
        return None


Single scraper

In [ ]:

class SingleScraper :
  #scrape single job post
  def __init__(self, url, fields) :
    self.url = str(url)
    self.page = requests.get(self.url)
    if(self.page.status_code == 200) :
      self.soup = bs(self.page.content, "html.parser")
      self.row = []
      self.fields = fields
    else:
      print('Failed to retrieve the web page. Status code:', self.page.status_code)



  def scrape(self) :

    # The About of the offre (it takes both the offer divs those eli contains all the infos about the job post to use them after that)
    top_annonce = self.soup.find('div', attrs={'class' : 'top-annonce'})
    detail_offre = self.soup.find('div', attrs={'class' : re.compile(r'detail-offre(\snotfeatured)?')})

    #Start addding the about section of the job accessing first div[class="top-annonce"] > h1
    Poste = top_annonce.h1.text
    print("Emploi : " , Poste)
    self.row.append(translate_text(Poste,'fr'))
    # self.row.append(Classify().classify(Poste))


    # looking for the all the il in a list div[class="top-annonce"] > ul > il (eli houmma 3 )
    identity = top_annonce.ul.find_all('li')

    #company
    print("Entreprise   : ", identity[0].text)
    self.row.append(identity[0].text)

    #location
    print("Location  : ", identity[1].text)
    self.row.append(identity[1].text)

    #posted at
    print("Date de poste : ", identity[2].text)
    # self.row.append(self.toHour(identity[2].text.split()))
    self.row.append(identity[2].text)

    #candidates in a try a catch if there s a limit in for how many are needed
    candidates = top_annonce.find('div', attrs={'class' : 'applicants-num'})
    try:
      print("candidats : ", candidates.text)
      if candidates.text == '':
        self.row.append(0)
      else:
        self.row.append(candidates.text)
    except:
      # if its empty the it feel it with "none"
      print("candidats : 0")
      self.row.append(0)

    # look for div[class="infos_job_details"] which is inside of div[class="detail-offre"]
    detail = detail_offre.find('div', attrs={'class' : 'infos_job_details'})

    #Retrieve all the divs inside of it
    details = detail.find_all('div')
    last = 0
    for i in range(len(details)-1) :
      label = (details[i].dl.dt.text).split()[0]
      value = details[i].dl.dd.text
      for j in range(last, len(self.fields)) :
        if self.fields[j] == label :
          print(label , ' : ', value)
          # if i == 0: self.row.append(value[0])
          # else : self.row.append(value)
          self.row.append(value)
          last = j+1
          break
        self.row.append('None')
        print(self.fields[j] , ' : ', None)

    for k in range(last, len(self.fields)):
      print(self.fields[k] , ' : None')
      self.row.append('None')

    print("*********************************************************************")
    return self.row


global scraper

In [ ]:
class GlobalScraper:
    def __init__(self, url, pageNum, filename, header):
      self.url = str(url)
      self.page = requests.get(self.url)
      if(self.page.status_code == 200) :
        self.pageNum = int(pageNum)
        self.filename = filename
        self.CsvWriter = Writer(filename)
        # self.CsvWriter.write_to_csv(header)
        self.soup = bs(self.page.content, "html.parser")
        self.prefixPageUrl = self.soup.find('span', class_='pad_right_small').a['href']
        self.prefixPageUrl = self.prefixPageUrl[:-1]
        self.fields = ["Postes", "Type", "Experience", "Niveau", "Rémunération", "Langue", "Genre"] # for single scrapper
      else:
        print('Failed to retrieve the web page. Status code:', self.page.status_code)

    def scrape(self, num):
        if num <= self.pageNum:
            pageUrl = str(self.url) + str(self.prefixPageUrl) + str(num)
            print(pageUrl)
            self.page = requests.get(pageUrl)
            if self.page.status_code == 200:
                self.soup = bs(self.page.content, "html.parser")
                posts = self.soup.find_all('article', class_="media")
                for post in posts:
                    postUrl = post.find('a')['href']
                    row = SingleScraper(postUrl, self.fields).scrape()
                    self.CsvWriter.write_to_csv(row)
            else:
                print('Failed to retrieve the web page. Status code:', self.page.status_code)

            self.scrape(num + 1)


Execution

In [ ]:
URL = "https://www.tanitjobs.com/jobs/"
MAXPAGESNUM = 1
FILENAME = 'finalData.csv'
HEADER = ["Position", "Entreprise", "Location", "Date", "Candidates", "Postes", "Type", "Experience", "Niveau", "Rémunération", "Langue", "Genre"]

#running test
execute = GlobalScraper(URL, MAXPAGESNUM, FILENAME, HEADER)
execute.scrape(1)


**Unitest**

In [ ]:
import unittest
from unittest.mock import MagicMock, patch

# Define your SingleScraper class and other necessary code here

class TestSingleScraper(unittest.TestCase):

    def setUp(self):
        # Set up common test data or mock objects
        self.url = "https://www.tanitjobs.com/jobs/?searchId=1705315644.0337&action=search&page=1"
        self.fields = ["Postes", "Type", "Experience", "Niveau", "Rémunération", "Langue", "Genre"]
        self.header = ["Position", "Entreprise", "Location", "Date", "Candidates"] + self.fields
        self.scraper = SingleScraper(self.url, self.fields)

    def test_scrape_specific_url(self):
        # Test the scrape method with a specific URL
        with patch('requests.get') as mock_get:
            # Mock the response for the specific URL
            mock_get.return_value.status_code = 200
            mock_get.return_value.content = b'<html>YourMockedHTMLContent</html>'

            # Create a BeautifulSoup mock object
            mock_soup = MagicMock()
            with patch('bs4.BeautifulSoup', return_value=mock_soup):
                # Call the scrape method
                result = self.scraper.scrape()

                # Add assertions based on the expected behavior of the scrape method
                self.assertIsNotNone(result)
                self.assertEqual(len(result), len(self.header))
                # Add more assertions based on your specific case

# Run the tests
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestSingleScraper))


E
ERROR: test_scrape_specific_url (__main__.TestSingleScraper)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-12-3cf984c7391f>", line 26, in test_scrape_specific_url
    result = self.scraper.scrape()
  File "<ipython-input-5-2edebe83cbdd>", line 22, in scrape
    Poste = top_annonce.h1.text
AttributeError: 'NoneType' object has no attribute 'h1'

----------------------------------------------------------------------
Ran 1 test in 6.664s

FAILED (errors=1)


<unittest.runner.TextTestResult run=1 errors=1 failures=0>